# Lean Security ID Logic

In [1]:
import sys
import os
from datetime import datetime
from numpy import base_repr
os.chdir(r'D:\REPOS\LeanVersions\Lean\Launcher\bin\Debug')

## First, let's import the Lean assemblies into Python, so we can compare the results

In [2]:
from clr import AddReference
AddReference("QuantConnect.Common")

from QuantConnect.Data import BaseData
from QuantConnect.Data.Auxiliary import LocalDiskMapFileProvider
from QuantConnect.Util import Composer
from QuantConnect.Interfaces import IMapFileProvider
from QuantConnect import SecurityIdentifier, Symbol, SecurityType, Market

Let's create a equity identifier and check it value. 

In [3]:
spy_date = datetime(1998, 1, 2)   # the origin of this date will be explain later, for now, take it as given.
sid = SecurityIdentifier.GenerateEquity(spy_date, "SPY", Market.USA)
str(sid)

'SPY R735QTJ8XC9X'

The SecurityIdentigfier objects has to parts, the first is the familiar ticker, and the generated *ID*. 

The ID is calculated by encoding of the security type, the market and the date defined above. For secutities like Option and Futures, the ID calulation also includes expiry, strike, rights (call, put) etc.

In order to transform the categories (markets, types) to integers, some constats are defined:

In [4]:
security_type_width = 100
security_type_offset = 1;
market_width = 1000;
market_offset = security_type_offset * security_type_width;
days_offset = 100000000000000

#### Security Type and Market encoding
SecurityType is an Enum, is has an integer value for each one of its _alternatives_ 

In [5]:
print (f'Equity -> {int(SecurityType.Equity)}')
print (f'Crypto -> {int(SecurityType.Crypto)}')
print (f'Forex -> {int(SecurityType.Forex)}')

Equity -> 1
Crypto -> 7
Forex -> 4


Market object is a class, but it has a similar behavior using the proper methods

In [6]:
print (f'USA -> {Market.Encode(Market.USA)}')
print (f'USA -> {Market.Encode("usa")}')
print (f'1 -> {Market.Decode(1)}\n')

print (f'Oanda -> {Market.Encode(Market.Oanda)}')
print (f'3 -> {Market.Decode(3)}')

USA -> 1
USA -> 1
1 -> usa

Oanda -> 3
3 -> oanda


#### Date Encoding
The date econding is simply the [OLE Automation representation](https://msdn.microsoft.com/en-us/library/system.datetime.tooadate.aspx) of the date defined above.

In order to reproduce the conversion, we'll use the following method [(source)](https://stackoverflow.com/a/10443946/3928185) 

In [7]:
def excel_date(date1):
    temp = datetime(1899, 12, 30, 0, 0, 0)    # Note, not 31st Dec but 30th!
    delta = date1 - temp
    return float(delta.days) + (float(delta.seconds) / 86400)

# Test cases from MSDN
assert(excel_date(datetime(1899, 12, 31))==1)
assert(excel_date(datetime(1900, 1, 1, 6, 0, 0))==2.25)
assert(excel_date(datetime(1899, 12, 29, 0, 0, 0))==-1)

#### Finally, the ID generation!

This is simple the encoding 

In [8]:
days = excel_date(spy_date) * days_offset
market_code = Market.Encode(Market.USA) * market_offset

In [9]:
base_repr(int(market_code + days), 36)

'R735QTJ8XC74'